In [ ]:
# study N_orb & N_r relation (demo)
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d
from utils.plot3dtree import plot3dtree

p_list = numpy.array([3,4,5,6])
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']
Nr_list = numpy.zeros((p_list.size,tol_list.size,len(basis_list)))

# loop over order
for ip in range(p_list.size):
  p = p_list[ip]
  # loop over tol
  for i in range(tol_list.size):
    tol = tol_list[i]
    # loop over d,t,qz
    for j in range(len(basis_list)):
      # basis
      mol_h2 = gto.M(
          verbose = 0,
          atom = '''
          h    0    0.       0.
          h    0    0.       0.74''',
          basis = basis_list[j]) 


      dom = numpy.array([-11, 11, -11, 11, -11, 11])
      checkpts = numpy.array([[ 0,    0.,       0.],
                              [ 0,    0.,    0.74]])

      ifcoeffs = False

      # tree
      def pyscffunc(x, y, z, mol):
        valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        nrows, ncols = valstmp.shape
        vals = numpy.zeros((nrows, ncols**2))
        for j in range(ncols):
          for k in range(ncols):
              vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
        return vals
      # def pyscffunc(x, y, z, mol):
      #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
      #   nrows, ncols = valstmp.shape
      #   vals = numpy.zeros((nrows))
      #   for j in range(ncols):
      #     vals = vals + valstmp[:, j]**2
      #   return vals
      func = lambda x, y, z: pyscffunc(x, y, z, mol_h2)
      tree_h2 = {
          'domain': dom.reshape(-1,1), 
          'tol': tol,
          'nSteps': 15,
          'n': p,
          'checkpts': checkpts.transpose(),
          'ifcoeffs': ifcoeffs                     
        }
      tree_h2, rint = buildBreadthFirst3d(tree_h2, func)
      Nr_list[ip][i][j] = p**3*numpy.sum(tree_h2['height']==0)

plot3dtree(tree_h2)

In [ ]:
p**3*numpy.sum(tree_h2['height']==0)
Nr_list[0,:,:],Nr_list[1,:,:],Nr_list[2,:,:],Nr_list[3,:,:]


In [ ]:
p_list = numpy.array([3,4,5,6])
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']
Nr_list = numpy.zeros((p_list.size,tol_list.size,len(basis_list)))

# loop over order
for ip in range(p_list.size):
  p = p_list[ip]
  # loop over tol
  for i in range(tol_list.size):
    tol = tol_list[i]
    # loop over d,t,qz
    for j in range(len(basis_list)):
      # study N_orb & N_r relation (demo)
      # basis
      mol_hf = gto.M(
          verbose = 0,
          atom = '''
          h    0    0.       0.
          f    0    0.       0.92''',
          basis = basis_list[j]) 


      dom = numpy.array([-13, 13, -13, 13, -13, 13])
      checkpts = numpy.array([[ 0,    0.,       0.],
                              [ 0,    0.,    0.92]])

      ifcoeffs = False

      # tree
      def pyscffunc(x, y, z, mol):
        valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        nrows, ncols = valstmp.shape
        vals = numpy.zeros((nrows, ncols**2))
        for j in range(ncols):
          for k in range(ncols):
              vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
        return vals
      # def pyscffunc(x, y, z, mol):
      #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
      #   nrows, ncols = valstmp.shape
      #   vals = numpy.zeros((nrows))
      #   for j in range(ncols):
      #     vals = vals + valstmp[:, j]**2
      #   return vals
      func = lambda x, y, z: pyscffunc(x, y, z, mol_hf)
      tree_hf = {
          'domain': dom.reshape(-1,1), 
          'tol': tol,
          'nSteps': 15,
          'n': p,
          'checkpts': checkpts.transpose(),
          'ifcoeffs': ifcoeffs                     
        }
      tree_hf, rint = buildBreadthFirst3d(tree_hf, func)
      Nr_list[ip][i][j] = p**3*numpy.sum(tree_hf['height']==0)

plot3dtree(tree_hf)

In [ ]:
p**3*numpy.sum(tree_hf['height']==0)
Nr_list[0,:,:],Nr_list[1,:,:],Nr_list[2,:,:],Nr_list[3,:,:]

In [ ]:
p_list = numpy.array([3,4,5,6])
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']

# loop over order
for ip in range(p_list.size):
  p = p_list[ip]
  # loop over tol
  for i in range(tol_list.size):
    tol = tol_list[i]
    # loop over d,t,qz
    for j in range(len(basis_list)):
      # study N_orb & N_r relation (demo)
      # basis
      mol_h2o = gto.M(
          verbose = 0,
          atom = '''
          o    0    0.       0.
          h    0    -0.757   0.587
          h    0    0.757    0.587''',
          basis = basis_list[j]) 


      dom = numpy.array([-15, 15, -15, 15, -15, 15])
      checkpts = numpy.array([[ 0,    0.,       0.],
                              [ 0,    -0.757,   0.587],
                              [ 0,    0.757,    0.587]])

      ifcoeffs = False

      # tree
      def pyscffunc(x, y, z, mol):
        valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        nrows, ncols = valstmp.shape
        vals = numpy.zeros((nrows, ncols**2))
        for j in range(ncols):
          for k in range(ncols):
              vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
        return vals
      # def pyscffunc(x, y, z, mol):
      #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
      #   nrows, ncols = valstmp.shape
      #   vals = numpy.zeros((nrows))
      #   for j in range(ncols):
      #     vals = vals + valstmp[:, j]**2
      #   return vals
      func = lambda x, y, z: pyscffunc(x, y, z, mol_h2o)
      tree_h2o = {
          'domain': dom.reshape(-1,1), 
          'tol': tol,
          'nSteps': 15,
          'n': p,
          'checkpts': checkpts.transpose(),
          'ifcoeffs': ifcoeffs                     
        }
      tree_h2o, rint = buildBreadthFirst3d(tree_h2o, func)
      Nr_list[ip][i][j] = p**3*numpy.sum(tree_h2o['height']==0)

plot3dtree(tree_h2o)

In [ ]:
p**3*numpy.sum(tree_h2o['height']==0)
Nr_list[0,:,:],Nr_list[1,:,:],Nr_list[2,:,:],Nr_list[3,:,:]

In [ ]:
p_list = numpy.array([3,4,5,6])
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']
Nr_list = numpy.zeros((p_list.size,tol_list.size,len(basis_list)))

# loop over order
for ip in range(p_list.size):
  p = p_list[ip]
  # loop over tol
  for i in range(tol_list.size):
    tol = tol_list[i]
    # loop over d,t,qz
    for j in range(len(basis_list)):
      # study N_orb & N_r relation (demo)
      # basis
      mol_ch4 = gto.Mole()
      mol_ch4.atom = [ # CH4
          ['H',    0.0000,    0.9870,    0.0000],
          ['C',    1.0390,    0.9870,    0.0000],
          ['H',    1.0390,    1.9740,    0.0000],
          ['H',    1.0390,    0.0000,    0.0000],
          ['H',    2.0260,    0.9870,    0.0000]]
      mol_ch4.basis = {'H': basis_list[j],
                      'C': basis_list[j],}
      mol_ch4.build()


      dom = numpy.array([-15, 15, -15, 15, -15, 15])
      checkpts = numpy.array([[ 0.0000,    0.9870,    0.0000],
                              [ 1.0390,    0.9870,    0.0000],
                              [ 1.0390,    1.9740,    0.0000],
                              [ 1.0390,    0.0000,    0.0000],
                              [ 2.0260,    0.9870,    0.0000]])

      ifcoeffs = False

      # tree
      def pyscffunc(x, y, z, mol):
        valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        nrows, ncols = valstmp.shape
        vals = numpy.zeros((nrows, ncols**2))
        for j in range(ncols):
          for k in range(ncols):
              vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
        return vals
      # def pyscffunc(x, y, z, mol):
      #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
      #   nrows, ncols = valstmp.shape
      #   vals = numpy.zeros((nrows))
      #   for j in range(ncols):
      #     vals = vals + valstmp[:, j]**2
      #   return vals
      func = lambda x, y, z: pyscffunc(x, y, z, mol_ch4)
      tree_ch4 = {
          'domain': dom.reshape(-1,1), 
          'tol': tol,
          'nSteps': 15,
          'n': p,
          'checkpts': checkpts.transpose(),
          'ifcoeffs': ifcoeffs                     
        }
      tree_ch4, rint = buildBreadthFirst3d(tree_ch4, func)
      Nr_list[ip][i][j] = p**3*numpy.sum(tree_ch4['height']==0)

plot3dtree(tree_ch4)

In [ ]:
p**3*numpy.sum(tree_ch4['height']==0)
Nr_list[0,:,:],Nr_list[1,:,:],Nr_list[2,:,:],Nr_list[3,:,:]